In [38]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)

Let's use the mads_datasets package (see [github](https://github.com/raoulg/mads_datasets) for more details) which I created for these lessons to give everyone easy access to the datasets we use for training.

In [39]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

for dataset in DatasetType:
    print(dataset)

DatasetType.FLOWERS
DatasetType.IMDB
DatasetType.GESTURES
DatasetType.FASHION
DatasetType.SUNSPOTS
DatasetType.IRIS
DatasetType.PENGUINS
DatasetType.FAVORITA
DatasetType.SECURE


There are a few datasets. For images, we can use FLOWERS (~3000 photos of flowers in 5 categories) and FASHION (60k fashion icons 28x28 pixels big).

Lets start with our good'ol MNIST.

In [40]:

fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-09-19 20:19:24.449 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\j.nagelhout\.cache\mads_datasets\fashionmnist
2025-09-19 20:19:24.450 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\j.nagelhout\.cache\mads_datasets\fashionmnist\fashionmnist.pt


We can obtain an item:

In [41]:
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

The image follows the channels-first convention: (channel, width, height). The label is an integer.

Let's re-use the model we had:

In [42]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cpu


In [43]:
from torch import nn
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self, filters, units1, units2, input_size=(32, 1, 28, 28)):
        super().__init__()
        self.in_channels = input_size[1]
        self.input_size = input_size
        self.filters = filters
        self.units1 = units1
        self.units2 = units2

        self.convolutions = nn.Sequential(
            nn.Conv2d(self.in_channels, filters, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(input_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, 10)
        )

    def _conv_test(self, input_size = (32, 1, 28, 28)):
        x = torch.ones(input_size)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits

model = CNN(filters=32, units1=128, units2=64).to("cpu")

2025-09-19 20:19:24.532 | INFO     | __main__:__init__:27 - Aggregating activationmap with size torch.Size([2, 2])


Using cpu device


In [44]:
from mltrainer.imagemodels import CNNConfig, CNNblocks

In [45]:
config = CNNConfig(
    matrixshape = (28, 28), # every image is 28x28
    batchsize = batchsize,
    input_channels = 1, # we have black and white images, so only one channel
    hidden = 32, # number of filters
    kernel_size = 3, # kernel size of the convolution
    maxpool = 3, # kernel size of the maxpool
    num_layers = 4, # we will stack 4 Convolutional blocks, each with two Conv2d layers
    num_classes = 10,
)

In [46]:
model = CNNblocks(config)
model.config

Calculated matrix size: 9
Caluclated flatten size: 288


{'matrixshape': (28, 28),
 'batchsize': 64,
 'input_channels': 1,
 'hidden': 32,
 'kernel_size': 3,
 'maxpool': 3,
 'num_layers': 4,
 'num_classes': 10}

In [47]:
from torchinfo import summary
summary(model, input_size=(32, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNNblocks                                [32, 10]                  --
├─ModuleList: 1-1                        --                        --
│    └─ConvBlock: 2-1                    [32, 32, 28, 28]          --
│    │    └─Sequential: 3-1              [32, 32, 28, 28]          9,568
│    └─ConvBlock: 2-2                    [32, 32, 28, 28]          --
│    │    └─Sequential: 3-2              [32, 32, 28, 28]          18,496
│    └─ReLU: 2-3                         [32, 32, 28, 28]          --
│    └─MaxPool2d: 2-4                    [32, 32, 9, 9]            --
│    └─ConvBlock: 2-5                    [32, 32, 9, 9]            --
│    │    └─Sequential: 3-3              [32, 32, 9, 9]            18,496
│    └─ReLU: 2-6                         [32, 32, 9, 9]            --
│    └─ConvBlock: 2-7                    [32, 32, 9, 9]            --
│    │    └─Sequential: 3-4              [32, 32, 9, 9]            18,496


And set up the optimizer, loss and accuracy.

In [48]:
import torch.optim as optim
from mltrainer import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [49]:
yhat = model(x.to("cpu"))
accuracy(y.to("cpu"), yhat)

0.078125

In [50]:
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="demo",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TOML],
)

In [51]:
trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
trainer.loop()

2025-09-19 20:19:24.653 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to demo\20250919-201924
2025-09-19 20:19:24.655 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:05<00:00, 18.96it/s]
2025-09-19 20:19:31.697 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.7664 test 0.9804 metric ['0.5995']
100%|██████████| 100/100 [00:04<00:00, 20.64it/s]
2025-09-19 20:19:38.236 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.8358 test 0.7311 metric ['0.6919']
100%|██████████| 100/100 [00:05<00:00, 19.65it/s]
2025-09-19 20:19:44.921 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.7208 test 0.6731 metric ['0.7488']
100%|██████████| 3/3 [00:20<00:00,  6.75s/it]


# MLflow
MLflow is an open-source platform designed to manage the entire Machine Learning (ML) lifecycle, including experimentation, reproducibility, deployment, and governance. It provides a set of APIs and tools to streamline ML workflows, making it easier to track experiments, package code, manage model versions, and deploy models.

Reasons to use MLflow over TensorBoard, gin-config, or Ray:

- End-to-end ML lifecycle management: While TensorBoard focuses on visualizing model training metrics and gin-config on hyperparameter configuration, MLflow covers a broader range of tasks, such as experiment tracking, model packaging, and deployment.

- Framework agnostic: MLflow is not tied to a specific ML framework, making it suitable for projects using different libraries or even multiple libraries.

- Model Registry: MLflow provides a centralized model registry, allowing you to version, track, and manage your models, which is not available in TensorBoard or gin-config.

- Deployment support: MLflow facilitates model deployment to various platforms, such as local, cloud, or Kubernetes environments, whereas TensorBoard and gin-config are not built for deployment tasks.

- Integration with other tools: MLflow integrates with popular tools and platforms like Databricks, AWS, and Azure, making it easy to incorporate into existing workflows.

However, the choice between MLflow and other tools like TensorBoard, gin-config, or Ray depends on your specific use case and the scope of the ML workflow you want to manage.

In [52]:
experiment_path = "mlflow_test"

In [53]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_path)

<Experiment: artifact_location='file:c:/GitHub/portfolio-JN/2-hypertuning-mlflow/mlruns/1', creation_time=1758304515015, experiment_id='1', last_update_time=1758304515015, lifecycle_stage='active', name='mlflow_test', tags={}>

In the code above, we set the MLflow tracking URI to a local SQLite database file. This is done to configure the storage location for MLflow's experiment tracking data, such as metrics, parameters, and artifacts. By specifying a SQLite database, we enable a lightweight and easy-to-use storage solution for tracking the experiments and their associated information.

The line mlflow.set_experiment("mnist_convolutions") sets the active MLflow experiment to "mnist_convolutions". This is useful for organizing and grouping your runs, as it allows you to associate the upcoming ML training runs with a specific experiment name, making it easier to search, compare, and analyze the results later.

In [54]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

We import functions and classes from the hyperopt library to perform hyperparameter optimization. This library helps us find the best hyperparameter values for our machine learning model by searching through a defined search space and using optimization algorithms like Tree-structured Parzen Estimator (TPE). The goal is to improve our model's performance by tuning its hyperparameters.

Advantages of TPE:

- Model-based approach: TPE is a Bayesian optimization method that models the objective function as a probability distribution. It learns from previous evaluations to decide which points in the search space to explore next, making it more efficient in finding optimal hyperparameters.

- Exploration-exploitation trade-off: TPE balances the trade-off between exploration (searching in new regions of the search space) and exploitation (refining around the current best points). This can lead to better results in problems with complex search spaces.

- Continuous hyperparameter optimization: TPE can handle continuous hyperparameters more naturally, as it builds a probability model to estimate the performance for any given point in the search space.

Lets set up an objective function and start logging some usefull things we might want to track:

In [55]:
modeldir = Path("models").resolve()
if not modeldir.exists():
    modeldir.mkdir()
    print(f"Created {modeldir}")

In [56]:
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime

# Define the hyperparameter search space
settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir=modeldir,
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW, ReportTypes.TOML],
)


# Define the objective function for hyperparameter optimization
def objective(params):
    # Start a new MLflow run for tracking the experiment
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "raoul")
        # Log hyperparameters to MLflow
        mlflow.log_params(params)
        mlflow.log_param("batchsize", f"{batchsize}")


        # Initialize the optimizer, loss function, and accuracy metric
        optimizer = optim.Adam
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()
        config = CNNConfig(
            matrixshape = (28, 28), # every image is 28x28
            batchsize = batchsize,
            input_channels = 1, # we have black and white images, so only one channel
            hidden = params["filters"], # number of filters
            kernel_size = 3, # kernel size of the convolution
            maxpool = 3, # kernel size of the maxpool
            num_layers = 4, # we will stack 4 Convolutional blocks, each with two Conv2d layers
            num_classes = 10,
        )

        # Instantiate the CNN model with the given hyperparameters
        model = CNNblocks(config)
        # Train the model using a custom train loop
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        # Save the trained model with a timestamp
        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

        # Log the saved model as an artifact in MLflow
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : trainer.test_loss, 'status': STATUS_OK}

See https://hyperopt.github.io/hyperopt/getting-started/search_spaces/ for more information about searchspaces for hyperopt

In [57]:
search_space = {
    'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
    'kernel_size' : scope.int(hp.quniform('kernel_size', 2, 5, 1)),
    'num_layers' : scope.int(hp.quniform('num_layers', 1, 10, 1)),
}

We define a search space for hyperparameter optimization using Hyperopt. The search space specifies the range and distribution of hyperparameters to explore during the optimization process. This is crucial for finding the optimal set of hyperparameters that yield the best performance for the machine learning model. The search space defined here includes the number of filters in the convolutional layers, and the number of units in two fully connected layers, allowing Hyperopt to find the best combination within the given ranges.


Now, finally, let us perform the hyperparameter search using the fmin function from hyperopt. The function takes the following arguments:

- `fn=objective`: The objective function to minimize, which is defined earlier to train the model and return the test loss.
- `space=search_space`: The search space defined earlier, containing the range of hyperparameters to explore.
- `algo=tpe.suggest`: The optimization algorithm to use, in this case, the Tree-structured Parzen Estimator (TPE) method.
- `max_evals=10`: The maximum number of function evaluations, i.e., the maximum number of hyperparameter combinations to try.
- `trials=Trials()`: A Trials object to store the results of each evaluation.

The fmin function searches for the best hyperparameters within the given search space using the TPE algorithm, aiming to minimize the objective function (test loss). Once the optimization process is completed, the best hyperparameters found are stored in the best_result variable.

In [58]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

Calculated matrix size: 9                            
Caluclated flatten size: 648                         
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

2025-09-19 20:19:45.059 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to C:\GitHub\portfolio-JN\2-hypertuning-mlflow\models\20250919-201945
2025-09-19 20:19:45.062 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:16<00:00,  6.01it/s]
2025-09-19 20:20:07.304 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.4897 test 0.8885 metric ['0.6767']
100%|##########| 100/100 [00:16<00:00,  6.11it/s]
2025-09-19 20:20:30.194 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.7740 test 0.7282 metric ['0.7314']
100%|##########| 100/100 [00:17<00:00,  5.74it/s]
2025-09-19 20:20:53.450 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.6668 test 0.6430 metric ['0.7580']
100%|##########| 3/3 [01:08<00:00, 22.79s/it]


Calculated matrix size: 9                                                      
Caluclated flatten size: 432                                                   
 33%|███▎      | 1/3 [01:08<02:16, 68.48s/trial, best loss: 0.6430426985025406]

2025-09-19 20:20:53.522 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to C:\GitHub\portfolio-JN\2-hypertuning-mlflow\models\20250919-202053
2025-09-19 20:20:53.524 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:09<00:00, 10.41it/s]
2025-09-19 20:21:06.168 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.4218 test 0.7992 metric ['0.6969']
100%|##########| 100/100 [00:09<00:00, 10.28it/s]
2025-09-19 20:21:19.070 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.7466 test 0.6999 metric ['0.7369']
100%|##########| 100/100 [00:09<00:00, 10.74it/s]
2025-09-19 20:21:31.428 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.6765 test 0.6708 metric ['0.7473']
100%|##########| 3/3 [00:37<00:00, 12.63s/it]


Calculated matrix size: 9                                                      
Caluclated flatten size: 936                                                   
 67%|██████▋   | 2/3 [01:46<00:50, 50.53s/trial, best loss: 0.6430426985025406]

2025-09-19 20:21:31.487 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to C:\GitHub\portfolio-JN\2-hypertuning-mlflow\models\20250919-202131
2025-09-19 20:21:31.488 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:33<00:00,  2.99it/s]
2025-09-19 20:22:14.713 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.5941 test 0.8826 metric ['0.6323']
100%|##########| 100/100 [00:35<00:00,  2.81it/s]
2025-09-19 20:23:00.999 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.7763 test 0.7264 metric ['0.7086']
100%|##########| 100/100 [00:33<00:00,  3.01it/s]
2025-09-19 20:23:44.451 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.6365 test 0.6685 metric ['0.7542']
100%|##########| 3/3 [02:12<00:00, 44.32s/it]


100%|██████████| 3/3 [03:59<00:00, 79.83s/trial, best loss: 0.6430426985025406]


After running this, you can look at the best_result

In [59]:
best_result

{'filters': np.float64(72.0),
 'kernel_size': np.float64(3.0),
 'num_layers': np.float64(3.0)}

# MLflow GUI
MLflow has a really great dashboard.
you can see it with the command:
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
```

The `--backend-store-uri` argument specifies the location of the SQLite database file, which is used to store experiment metadata, such as parameters, metrics, and artifacts. 
We have initialized the experiment with `mlflow.set_tracking_uri("sqlite:///mlflow.db")`, so `mlflow.db` is the location we need. 

`--host` tells the server we use our own machine (localhost), and `--port` specifies the port number on which the server will listen for incoming requests. In this case, we are using port 5000. Sometimes, you can have conflicts on a specific port and it could help to change the port (eg to 5001)

Note that on a windows machine, the `\` gives errors (because, why not, right) so for windows you might need to remove the `\` and put everything on a single line.

I have created a `Makefile` to automate these commands, but if it doesnt work (again, because you are on windows for example) you can just type the command by hand in your terminal.

After starting this up, go to `http://127.0.0.1:5000` in your browser. You should see the MLflow UI, where you can explore your experiments, runs, and metrics. The UI provides a user-friendly way to visualize and compare different runs, making it easier to analyze the results of your hyperparameter optimization and model training.

Test for tuning parameters


In [60]:
import torch
import torch.nn as nn
from typing import List, Literal, Optional

class FlexCNN(nn.Module):
    """
    Een modulaire CNN waarmee je conv-blokken, normalisatie en dropout als hyperparameters kunt variëren.
    """
    def __init__(
        self,
        input_channels: int = 1,
        num_classes: int = 10,
        conv_blocks: int = 3,
        base_filters: int = 32,
        filters_growth: float = 1.0,  # 1.0 = constant, 2.0 = verdubbelen per blok, etc.
        kernel_size: int = 3,
        pool_kernel: int = 2,
        norm: Literal["batch", "layer", "none"] = "batch",
        hidden_units: List[int] = [128, 64],
        dropout_p: float = 0.2,
        global_pool: Literal["avg", "max", "adaptive"] = "adaptive",
    ):
        super().__init__()
        self.input_channels = input_channels
        self.num_classes = num_classes
        self.conv_blocks = conv_blocks
        self.base_filters = base_filters
        self.filters_growth = filters_growth
        self.kernel_size = kernel_size
        self.pool_kernel = pool_kernel
        self.norm = norm
        self.hidden_units = hidden_units
        self.dropout_p = dropout_p
        self.global_pool = global_pool

        # ---- Convolutionele blokken (ModuleList) ----
        in_ch = input_channels
        convs = []
        filters = base_filters
        padding = kernel_size // 2  # 'same-ish' bij stride=1

        for b in range(conv_blocks):
            block = []
            block.append(nn.Conv2d(in_ch, int(filters), kernel_size=kernel_size, stride=1, padding=padding))
            if norm == "batch":
                block.append(nn.BatchNorm2d(int(filters)))
            elif norm == "layer":
                # LayerNorm over (C,H,W) -> channels-last LN: beter via GroupNorm met 1 group
                block.append(nn.GroupNorm(1, int(filters)))
            block.append(nn.ReLU(inplace=True))
            block.append(nn.MaxPool2d(kernel_size=pool_kernel))

            convs.append(nn.Sequential(*block))
            in_ch = int(filters)
            filters = int(filters * filters_growth if filters_growth > 0 else filters)

        self.convs = nn.ModuleList(convs)

        # ---- Global pooling om feature-dim te fixeren ----
        if global_pool == "avg":
            self.global_pool_layer = nn.AdaptiveAvgPool2d((1,1))
        elif global_pool == "max":
            self.global_pool_layer = nn.AdaptiveMaxPool2d((1,1))
        else:
            self.global_pool_layer = nn.AdaptiveAvgPool2d((1,1))  # default

        # ---- Dense gedeelte + Dropout ----
        dense_layers = [nn.Flatten()]  # (N, C, 1, 1) -> (N, C)
        in_features = in_ch  # na global pool is C over
        for hu in hidden_units:
            dense_layers.append(nn.Linear(in_features, hu))
            # Normalisatie kan ook op dense: BatchNorm1d of LayerNorm
            if norm == "batch":
                dense_layers.append(nn.BatchNorm1d(hu))
            elif norm == "layer":
                dense_layers.append(nn.LayerNorm(hu))
            dense_layers.append(nn.ReLU(inplace=True))
            if dropout_p and dropout_p > 0:
                dense_layers.append(nn.Dropout(p=dropout_p))
            in_features = hu

        dense_layers.append(nn.Linear(in_features, num_classes))
        self.dense = nn.Sequential(*dense_layers)

    def forward(self, x):
        for block in self.convs:
            x = block(x)
        x = self.global_pool_layer(x)
        logits = self.dense(x)
        return logits


In [61]:
import mlflow
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
import torch.optim as optim

# Tracking en experiment instellen
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("cnn_hparam_search")

# Trainer settings (pas aan op jouw dataset/pipeline)
accuracy = metrics.Accuracy()
settings = TrainerSettings(
    epochs=6,                  # iets langer trainen geeft stabielere vergelijking
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=200,           # pas aan naar je dataset/generator
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW, ReportTypes.TOML],
)


2025/09/19 20:37:14 INFO mlflow.tracking.fluent: Experiment with name 'cnn_hparam_search' does not exist. Creating a new experiment.
2025-09-19 20:37:14.486 | INFO     | mltrainer.settings:check_path:60 - Created logdir c:\GitHub\portfolio-JN\2-hypertuning-mlflow\modellogs


In [62]:
# Voorbeeld search space (pas gerust aan of breid uit)
grids = {
    "conv_blocks": [2, 3, 4],
    "base_filters": [16, 32],
    "filters_growth": [1.0, 2.0],
    "norm": ["none", "batch", "layer"],
    "dropout_p": [0.0, 0.3],
    "hidden_units": [(128,), (128, 64)],
    "lr": [1e-3, 3e-4],
    "weight_decay": [0.0, 1e-4],
    "batchsize": [64],  # als je batchsize varieert, log die ook en geef hem door aan je dataloader
}

import itertools

def run_one(hp):
    # Model maken
    model = FlexCNN(
        input_channels=1,
        num_classes=10,
        conv_blocks=hp["conv_blocks"],
        base_filters=hp["base_filters"],
        filters_growth=hp["filters_growth"],
        norm=hp["norm"],
        hidden_units=list(hp["hidden_units"]),
        dropout_p=hp["dropout_p"],
        global_pool="adaptive",
    ).to("cpu")

    # Trainer klaarzetten (je eigen streamers invullen)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam
    scheduler = None  # of bijvoorbeeld optim.lr_scheduler.ReduceLROnPlateau

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optimizer,
        traindataloader=trainstreamer,     # <-- gebruik je bestaande
        validdataloader=validstreamer,     # <-- gebruik je bestaande
        scheduler=scheduler
    )

    # MLflow run
    with mlflow.start_run():
        # Hyperparameters loggen
        mlflow.log_params({
            "conv_blocks": hp["conv_blocks"],
            "base_filters": hp["base_filters"],
            "filters_growth": hp["filters_growth"],
            "norm": hp["norm"],
            "dropout_p": hp["dropout_p"],
            "hidden_units": "-".join(map(str, hp["hidden_units"])),
            "lr": hp["lr"],
            "weight_decay": hp["weight_decay"],
            "batchsize": hp["batchsize"],
        })
        mlflow.set_tag("model", "FlexCNN")
        mlflow.set_tag("dev", "JN")

        # Zorg dat je optimizer de lr/weight_decay mee krijgt (je Trainer kan dat als class doorgeven)
        # Als je Trainer explicit kwargs verwacht:
        trainer.optimizer_kwargs = {"lr": hp["lr"], "weight_decay": hp["weight_decay"]}

        # Trainen/valideren
        trainer.loop()  # jouw Trainer logt MLFLOW/TOML al via settings

        # (Optioneel) extra metrics of artefacten loggen
        # mlflow.log_metric("final_val_acc", float(...))
        # mlflow.log_artifact("path/naar/confusion_matrix.png")
    return

# Bruteforce grid (klein houden om te starten)
keys = list(grids.keys())
values = [grids[k] for k in keys]
for combo in itertools.product(*values):
    hp = dict(zip(keys, combo))
    run_one(hp)


2025-09-19 20:38:19.175 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250919-203819
2025-09-19 20:38:19.178 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 200/200 [00:01<00:00, 118.67it/s]
2025-09-19 20:38:21.324 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.9764 test 1.6079 metric ['0.4353']
100%|██████████| 200/200 [00:01<00:00, 122.55it/s]
2025-09-19 20:38:23.383 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.3603 test 1.1987 metric ['0.5505']
100%|██████████| 200/200 [00:01<00:00, 132.17it/s]
2025-09-19 20:38:25.291 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.1248 test 1.0968 metric ['0.6255']
100%|██████████| 200/200 [00:01<00:00, 121.80it/s]
2025-09-19 20:38:27.385 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.0167 test 1.0335 metric ['0.6384']
100%|██████████| 200/200 [00:01<00:00, 128.77it/s]
2025-0